In [1]:
import pandas as pd
import feather


from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.models import load_model


Using TensorFlow backend.


In [2]:
n_groups_per_col = 9
architecture = "2_dense_512"
spread = "spread"
path = "../data/models/ind/categorical/" + architecture +"/" + spread + "/"

data_train = pd.read_feather(path + "data_train.feather").to_numpy()
labels_train = pd.read_feather(path + "labels_train.feather")

In [4]:
def fit_ind_categorical_model(data_train, labels_train):
    
    model = Sequential()
    model.add(Dense(units=512, activation='relu', input_shape=(data_train.shape[1],)))
    model.add(Dense(units=512, activation='relu'))
    model.add(Dense(units=512, activation='relu'))


    model.add(Dense(n_groups_per_col, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(data_train, labels_train, validation_split=0.2, batch_size=512, epochs=3, verbose=1)
    
    return (history, model)

In [5]:
# low
labels_low_train = to_categorical(labels_train.low.to_numpy(), n_groups_per_col)
(low_history, low_model) = fit_ind_categorical_model(data_train, labels_low_train)
feather.write_dataframe(pd.DataFrame.from_dict(low_history.history), path + "low_history.feather", compression="uncompressed")
low_model.save(path + "low_model.hdf5")


Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 34s 10us/step - loss: 2.3002 - accuracy: 0.1156 - val_loss: 2.5527 - val_accuracy: 0.1114
Epoch 2/3
3566135/3566135 [==============================] - 34s 9us/step - loss: 2.1974 - accuracy: 0.1111 - val_loss: 2.1973 - val_accuracy: 0.1114
Epoch 3/3
3566135/3566135 [==============================] - 34s 9us/step - loss: 2.1973 - accuracy: 0.1110 - val_loss: 2.1973 - val_accuracy: 0.1104


In [6]:
# high
labels_high_train = to_categorical(labels_train.high.to_numpy(), n_groups_per_col)
(high_history, high_model) = fit_ind_categorical_model(data_train, labels_high_train)
feather.write_dataframe(pd.DataFrame.from_dict(high_history.history), path + "high_history.feather", compression="uncompressed")
high_model.save(path + "high_model.hdf5")

Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 34s 10us/step - loss: 2.2903 - accuracy: 0.1147 - val_loss: 2.1973 - val_accuracy: 0.1111
Epoch 2/3
3566135/3566135 [==============================] - 33s 9us/step - loss: 2.1973 - accuracy: 0.1111 - val_loss: 2.1973 - val_accuracy: 0.1111
Epoch 3/3
3566135/3566135 [==============================] - 33s 9us/step - loss: 2.1973 - accuracy: 0.1114 - val_loss: 2.1973 - val_accuracy: 0.1113


In [7]:
# close
labels_close_train = to_categorical(labels_train.close.to_numpy(), n_groups_per_col)
(close_history, close_model) = fit_ind_categorical_model(data_train, labels_close_train)
feather.write_dataframe(pd.DataFrame.from_dict(close_history.history), path + "close_history.feather", compression="uncompressed")
close_model.save(path + "close_model.hdf5")

Train on 3566135 samples, validate on 891534 samples
Epoch 1/3
3566135/3566135 [==============================] - 33s 9us/step - loss: 2.2858 - accuracy: 0.1148 - val_loss: 2.1972 - val_accuracy: 0.1114
Epoch 2/3
3566135/3566135 [==============================] - 34s 9us/step - loss: 2.1979 - accuracy: 0.1112 - val_loss: 2.1973 - val_accuracy: 0.1109
Epoch 3/3
3566135/3566135 [==============================] - 33s 9us/step - loss: 2.1973 - accuracy: 0.1110 - val_loss: 2.1972 - val_accuracy: 0.1111


In [8]:
### Predict
# low_model = load_model(path + "low_model.hdf5")
# high_model = load_model(path + "high_model.hdf5")
# close_model = load_model(path + "close_model.hdf5")

In [9]:
data_test = pd.read_feather(path + "data_test.feather").to_numpy()

low_pred_prob = low_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(low_pred_prob), path + "low_pred_prob.feather", compression='uncompressed')

high_pred_prob = high_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(high_pred_prob), path + "high_pred_prob.feather", compression='uncompressed')

close_pred_prob = close_model.predict_proba(data_test, batch_size = 512)
feather.write_dataframe(pd.DataFrame(close_pred_prob), path + "close_pred_prob.feather", compression='uncompressed')
